In [144]:
import attribute
from datetime import datetime as dt
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from stock_exchange import create_interval_tree
from IPython.display import display, HTML


In [59]:
attributes = pd.read_csv("data/subset_attributes.csv", index_col=False, names=["name", "values", "comparetype", "up", "down"
])

temp_brain = {}

for i, row in attributes.iterrows():
    temp_brain[row['name']] = (0, 0)
    values = row['values'].split(',')
    values = list(map(str.strip, values))
    attributes.set_value(i,'values',values)
    
products = pd.read_csv("data/products_pivoted.csv", dtype=str, index_col="id")
match_data = pd.read_csv("data/matches.csv", index_col=False, usecols=[0, 1, 2], names=['original_id', 'replacement_id', 'timestamp'])
match_data['timestamp'] = pd.to_datetime(match_data['timestamp'])

matches_train, matches_test = train_test_split(match_data, test_size=0.3, random_state=42)

stock_exchange = create_interval_tree("data/stock_exchange.csv")


In [60]:
def equal(a, b):
    return 1 if a == b else 0

def greater_or_equal(a, b):
    return 1 if a <= b else 0

def lesser_or_equal(a, b):
    return 1 if a >= b else 0

def interval_match(a, b, up, down):
    if b <= a + up and b >= a - down:
        return 1
    else:
        return 0

def linear_interval_match(a, b, up, down):

    if b <= a + up and b >= a - down:
        if b > a and up > 0:
            return 1 - (b - a)* 0.5 / up
        elif a > b and down > 0:
            return 1.0 - (a - b) * 0.5 / down
        else:
            return 1.0
    else:
        return 0.0


In [61]:
def evaluate(a, b, attribute):
    compare_type = attribute['comparetype']
    
    if type(a) is float or type(b) is float:
        return (False, 0)
    else:
        a_index = attribute['values'].index(a)
        b_index = attribute['values'].index(b)
    
        if compare_type == 1:
            return (True, equal(a_index, b_index))
        elif compare_type == 2:
            return (True, greater_or_equal(a_index, b_index))
        elif compare_type == 3:
            return (True, lesser_or_equal(a_index, b_index))
        elif compare_type == 6:
            return (True, interval_match(a_index, b_index, attribute['up'], attribute['down']))
        elif compare_type == 50:
            return (True, linear_interval_match(a_index, b_index, attribute['up'], attribute['down']))
        else:
            return (False, 0)

In [62]:
def train(train_data):
    for _, match in matches_train.iterrows():
        original, replacement = products.loc[match['original_id']], products.loc[match['replacement_id']]
        timestamp = match['timestamp']
        for i, attribute in attributes.iterrows():
            attribute_name = attribute['name']
            try: 
                score = evaluate(original[attribute_name], replacement[attribute_name], attribute)
            except ValueError:
                score = (False, 0)


            if score[0]:
                old_score = temp_brain[attribute_name]
                temp_brain[attribute_name] = (old_score[0] + 1, old_score[1] + score[1])
                
    for name, (x, y) in temp_brain.items():
        brain[name] = 0 if x == 0 else y / x


In [63]:
brain = {}
train(matches_train)

In [74]:
def score(original_id, replacement_id):
    result = 0
    original = products.loc[original_id]
    replacement = products.loc[replacement_id]
    

    for i, attribute in attributes.iterrows():
        attribute_name = attribute['name']
        try: 
            score = evaluate(original[attribute_name], replacement[attribute_name], attribute)[1]
        except ValueError:
            score = 0
            
        result += brain[attribute_name] * score
    return result

In [137]:
def find_all_matches(product_a_id, date):
    """Returns all products that matches product_a_id in descending order"""
    matches = []

    stock_products = set(map((lambda x: x.data), stock_exchange[date]))

    for product_b_id in stock_products:

        match_score = score(product_a_id, product_b_id)
        matches.append((match_score, product_b_id))

    matches.sort(key=lambda x: -x[0])

    return [x[1] for x in matches]

In [140]:
def positions(test_data):
    """Returns the position that a match was found at given matches"""
    result = []

    for _, match in test_data.iterrows():
        original, replacement = match['original_id'], match['replacement_id']
        timestamp = match['timestamp']
        
        
        if timestamp < dt(2014, 6, 1, 12, 0, 0):
            next
        else:
            matches = find_all_matches(original, timestamp)
            index = matches.index(replacement)
            print("%s - (%s, %s)  -  %s" % (index,
                                     products.loc[original]['name'], 
                                     products.loc[replacement]['name'],
                                     products.loc[matches[0]]['name']
                                     ))
            if index > 5:
                display(products.loc[[original, replacement, matches[0]]])

In [145]:
positions(matches_test)

0 - (Xperia Z3, Xperia Z3)  -  Xperia Z3
0 - (One S, One Mini)  -  One Mini
0 - (iPhone 5s 32GB, iPhone 5s 32GB)  -  iPhone 5s 32GB
3 - (Galaxy S4 i9505 16GB, Galaxy S4 i9506 16GB)  -  Galaxy S4 i9505 16GB
0 - (iPhone 5s 32GB, iPhone 5s 32GB)  -  iPhone 5s 32GB
0 - (iPhone 6 16GB, iPhone 6 16GB)  -  iPhone 6 16GB
2 - (iPhone 4s 16GB, iPhone 5s 16GB)  -  iPhone 4s 8GB
0 - (iPhone 6 16GB, iPhone 6 16GB)  -  iPhone 6 16GB
0 - (iPhone 6 16GB, iPhone 6 16GB)  -  iPhone 6 16GB
0 - (iPhone 6 16GB, iPhone 6 16GB)  -  iPhone 6 16GB
0 - (iPhone 5s 16GB, iPhone 5s 16GB)  -  iPhone 5s 16GB
1 - (Galaxy S6 G920F 32GB, Galaxy S6 G920F 32GB)  -  Galaxy S6 Edge G925F 32GB
12 - (Ascend P7, P8)  -  Moto X 2 16GB


,name,producer,E-GSM,Bandtyp,WAP,Display,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,...,Qwerty-tangentbord,4G,Modell,Internminne,Batterikapacitet,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,
73775,Ascend P7,Huawei,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,...,Nej,Ja,NaN,2048,2500,Nej,Ja,Nej,FULL HD,Nej
82473,P8,Huawei,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,...,Nej,Ja,NaN,3072,2680,Nej,Ja,Nej,FULL HD,Nej
77561,Moto X 2 16GB,Motorola AB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,...,Nej,Ja,Snapdragon 801,2048,2300,Nej,Ja,Nej,FULL HD,Nej


0 - (iPhone 6 64GB, iPhone 6 64GB)  -  iPhone 6 64GB
0 - (iPhone 5s 16GB, iPhone 5s 16GB)  -  iPhone 5s 16GB
5 - (iPhone 6 16GB, iPhone 6s 16GB)  -  iPhone 6 16GB
0 - (iPhone 6 64GB, iPhone 6 64GB)  -  iPhone 6 64GB
0 - (iPhone 6 16GB, iPhone 6 16GB)  -  iPhone 6 16GB
0 - (Xperia Z1, Xperia Z1)  -  Xperia Z1
0 - (iPhone 6 64GB, iPhone 6 64GB)  -  iPhone 6 64GB
0 - (iPhone 5s 16GB, iPhone 5s 16GB)  -  iPhone 5s 16GB
0 - (iPhone 5 16GB, iPhone 5s 16GB)  -  iPhone 5s 16GB
0 - (Galaxy A3 A300F, Galaxy A3 A300F)  -  Galaxy A3 A300F
0 - (Galaxy S4 i9505 16GB, Galaxy S4 i9506 16GB)  -  Galaxy S4 i9506 16GB
0 - (G2 D802 16GB, G2 D802 16GB)  -  G2 D802 16GB
1 - (iPhone 4s 16GB, iPhone 5c 16GB)  -  iPhone 4s 8GB
6 - (iPhone 6 16GB, iPhone 6s 16GB)  -  iPhone 6 16GB


,name,producer,E-GSM,Bandtyp,WAP,Display,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,...,Qwerty-tangentbord,4G,Modell,Internminne,Batterikapacitet,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,...,Nej,Ja,A8,1024,1810,Ja,Nej,Nej,HD,Nej
85203,iPhone 6s 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,...,Nej,Ja,A8,2048,1715,Ja,Nej,Nej,HD,Nej
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,...,Nej,Ja,A8,1024,1810,Ja,Nej,Nej,HD,Nej


0 - (iPhone 6 64GB, iPhone 6 64GB)  -  iPhone 6 64GB
0 - (iPhone 5s 16GB, iPhone 5s 16GB)  -  iPhone 5s 16GB
0 - (iPhone 6 16GB, iPhone 6 16GB)  -  iPhone 6 16GB
0 - (iPhone 6 16GB, iPhone 6 16GB)  -  iPhone 6 16GB
0 - (Galaxy S3 i9300 16GB, Galaxy S3 i9300 16GB)  -  Galaxy S3 i9300 16GB
0 - (iPhone 6 16GB, iPhone 6 16GB)  -  iPhone 6 16GB
1 - (Galaxy S4 i9505 16GB, Galaxy S4 i9506 16GB)  -  Ascend P7
1 - (Galaxy S6 G920F 32GB, Galaxy S6 G920F 32GB)  -  Galaxy S6 Edge G925F 32GB
0 - (Xperia Z3, Xperia Z3)  -  Xperia Z3
1 - (Galaxy S4 i9505 16GB, Galaxy S4 i9506 16GB)  -  Ascend P7
1 - (Galaxy Mega i9205 8GB, S50)  -  Galaxy S4 Mini i9195 8GB
0 - (iPhone 5s 16GB, iPhone 5s 16GB)  -  iPhone 5s 16GB
1 - (iPhone 5c 16GB, iPhone 5s 16GB)  -  iPhone 5c 8GB
0 - (Xperia Z1, Xperia Z1)  -  Xperia Z1
0 - (Galaxy S3 i9305 16GB, Galaxy S3 i9305 16GB)  -  Galaxy S3 i9305 16GB
0 - (Xperia Z1, Xperia Z1)  -  Xperia Z1
0 - (Galaxy S7 G930F 32GB, Galaxy S7 G930F 32GB)  -  Galaxy S7 G930F 32GB
0 - (iPho

...

,name,producer,E-GSM,Bandtyp,WAP,Display,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,...,Qwerty-tangentbord,4G,Modell,Internminne,Batterikapacitet,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,...,Nej,Ja,A8,1024,1810,Ja,Nej,Nej,HD,Nej
85203,iPhone 6s 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,...,Nej,Ja,A8,2048,1715,Ja,Nej,Nej,HD,Nej
76645,iPhone 6 16GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,...,Nej,Ja,A8,1024,1810,Ja,Nej,Nej,HD,Nej


1 - (Xperia V, Xperia M4 Aqua)  -  Galaxy S4 Mini i9195 8GB
0 - (Galaxy Ace 4 G357F, Galaxy Ace 4 G357F)  -  Galaxy Ace 4 G357F
0 - (Xperia E4g, Xperia E4g)  -  Xperia E4g
1 - (iPhone 5 16GB, iPhone 5s 16GB)  -  iPhone 5c 16GB
0 - (Galaxy S4 i9505 16GB, Galaxy S4 Black Edition i9506 16GB)  -  Galaxy S4 Black Edition i9506 16GB
0 - (iPhone 6 Plus 128GB, iPhone 6 Plus 128GB)  -  iPhone 6 Plus 128GB
0 - (Galaxy S6 Edge+ G928F 32GB, Galaxy S6 Edge+ G928F 32GB)  -  Galaxy S6 Edge+ G928F 32GB
8 - (iPhone 5s 64GB, iPhone SE 64GB)  -  iPhone 5s 32GB


,name,producer,E-GSM,Bandtyp,WAP,Display,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,...,Qwerty-tangentbord,4G,Modell,Internminne,Batterikapacitet,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,
66267,iPhone 5s 64GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,...,Nej,Ja,A7,1024,1560,Ja,Nej,Nej,< HD,Nej
88043,iPhone SE 64GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,...,Nej,Ja,Nej,2048,1642,Ja,Ja,Nej,< HD,Nej
66274,iPhone 5s 32GB,Apple Sverige,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,...,Nej,Ja,A7,1024,1560,Ja,Nej,Nej,< HD,Nej


4 - (iPhone 6 16GB, iPhone 6s 32GB)  -  iPhone 6 16GB
2 - (iPhone 6 Plus 64GB, iPhone 6s Plus 64GB)  -  iPhone 6 Plus 128GB
0 - (iPhone 5s 16GB, iPhone 5s 16GB)  -  iPhone 5s 16GB
0 - (iPhone 5 16GB, iPhone 5c 16GB)  -  iPhone 5c 16GB
0 - (iPhone 5c 16GB, iPhone 5c 16GB)  -  iPhone 5c 16GB
0 - (iPhone 6 16GB, iPhone 6 16GB)  -  iPhone 6 16GB
0 - (iPhone 5s 16GB, iPhone 5s 16GB)  -  iPhone 5s 16GB
0 - (Galaxy Xcover 2 S7710, Galaxy Xcover 2 S7710)  -  Galaxy Xcover 2 S7710
0 - (iPhone 6 64GB, iPhone 6 64GB)  -  iPhone 6 64GB
0 - (Galaxy S4 i9505 16GB, Galaxy S4 Black Edition i9506 16GB)  -  Galaxy S4 Black Edition i9506 16GB


ValueError: 71988 is not in list

In [135]:

    
def visualize_score(original_id, replacement_id):
    original = products.loc[original_id]
    replacement = products.loc[replacement_id]
    total = 0
    print('')
    print("Comparing: %s with %s" % (original['name'], replacement['name']))
    
    for i, attribute in attributes.iterrows():
        attribute_name = attribute['name']
        try: 
            score = brain[attribute_name] * evaluate(original[attribute_name], replacement[attribute_name], attribute)[1]

        except ValueError:
            score = 0
                
        print("%s: %s - %s - %s - %s" %
                  (attribute_name, original[attribute_name], 
                   attribute['comparetype'], replacement[attribute_name], 
                   score))

        total += score


    print("Total score: %s" % total)
    print('')

In [136]:
#visualize_score(66267,88043)


Comparing: iPhone 5s 64GB with iPhone SE 64GB
Bandtyp: Quad-Band - 6 - Quad-Band - 0.9983515259523279
FM-Radio: Nej - 3 - Nej - 0.9791971612330894
Röststyrning: Ja - 1 - Ja - 0.9849023090586145
Bluetooth: Ja - 1 - Ja - 0.9990693551961002
Kamera: Ja - 1 - Ja - 0.9993231657792618
Java: Nej - 2 - Nej - 0.982481816569097
3G: Ja - 1 - Ja - 0.9958342565920674
USB: Nej - 1 - Nej - 0.9621845005332386
Max upplösning (stillbild): 3264x2448 - 6 - 3264x2448 - 0.9223817897616257
Max upplösning (video): 1920x1080 - 6 - 3840x2160 - 0.0
Storlek: 4 - 6 - 4 - 0.9682173295454546
Färgsupport: 16000000 - 6 - 16777216 - 0.9965732087227415
Bildskärmsteknik: nan - 2 - nan - 0.0
Telefondesign: Standard - 1 - Standard - 0.9976512297806337
Bildskärmsteknik: nan - 1 - nan - 0.0
Pixlar: nan - 6 - nan - 0
Lagringskapacitet: 64 - 50 - 64 - 0.9262942970959763
Trådlöst nätverk: Ja - 1 - Ja - 0.9990250387768669
Pekskärm: Ja - 1 - Ja - 0.999778378617969
Operativsystem: iOS - 6 - iOS - 0.9983152294391487
GPS: Ja - 1 - J